In [3]:
import pandas as pd

In [4]:
Dataset=pd.read_csv('CKD.csv')

In [5]:
Dataset.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [6]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [7]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [8]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
Dependent=Dataset[['classification_yes']]

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [10]:
from sklearn.preprocessing import StandardScaler
scx=StandardScaler()
X_train=scx.fit_transform(X_train)
X_test=scx.transform(X_test)

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],
    'p': [1, 2]
}


grid=GridSearchCV(KNeighborsClassifier(), param_grid, refit=True, n_jobs=-1)
grid.fit(X_train,Y_train)

c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
                         'leaf_size': [10, 20, 30, 40, 50],
                         'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15],
                         'p': [1, 2], 'weights': ['uniform', 'distance']})

In [12]:
re=grid.cv_results_

In [13]:
grid_Pred=grid.predict(X_test)

In [14]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(grid_Pred,Y_test)
print(cm)

[[45  2]
 [ 0 73]]


In [15]:
from sklearn.metrics import classification_report
clf_report=classification_report(grid_Pred,Y_test)
print(clf_report)

              precision    recall  f1-score   support

           0       1.00      0.96      0.98        47
           1       0.97      1.00      0.99        73

    accuracy                           0.98       120
   macro avg       0.99      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120



In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(grid_Pred,Y_test)
print("The f1 sore of the best parameter{}:".format(grid.best_params_),f1_score)
print(f1_macro)

The f1 sore of the best parameter{'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 1, 'p': 2, 'weights': 'uniform'}: <function f1_score at 0x0000014B00C53420>
0.9864864864864865


In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9866666666666667

In [19]:
table=pd.DataFrame.from_dict(re)
print(table)

     mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0         0.013210      0.017168         0.541308        0.015938   
1         0.005718      0.005124         0.325445        0.259221   
2         0.002893      0.001715         0.012455        0.011642   
3         0.003811      0.003567         0.004926        0.003548   
4         0.001208      0.001611         0.008060        0.004363   
..             ...           ...              ...             ...   
635       0.004106      0.001417         0.004393        0.000673   
636       0.002813      0.000534         0.004826        0.000944   
637       0.002275      0.001587         0.004456        0.001614   
638       0.004304      0.002182         0.005215        0.001303   
639       0.003744      0.001716         0.004604        0.001348   

    param_algorithm param_leaf_size param_n_neighbors param_p param_weights  \
0              auto              10                 1       1       uniform   
1            